# Training Doc2Vec on Wikipedia articles

This notebook replicates the **Document Embedding with Paragraph Vectors** paper, http://arxiv.org/abs/1507.07998.

In that paper, the authors only showed results from the DBOW ("distributed bag of words") mode, trained on the English Wikipedia. Here we replicate this experiment using not only DBOW, but also the DM ("distributed memory") mode of the Paragraph Vector algorithm aka Doc2Vec.

## Basic setup

Let's import the necessary modules and set up logging. The code below assumes Python 3.7+ and Gensim 4.0+.

In [1]:
import logging
import multiprocessing
from pprint import pprint

import smart_open
from gensim.corpora.wikicorpus import WikiCorpus, tokenize
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

## Preparing the corpus

First, download the dump of all Wikipedia articles from [here](http://download.wikimedia.org/enwiki/). You want the file named `enwiki-latest-pages-articles.xml.bz2`.

Second, convert that Wikipedia article dump from the arcane Wikimedia XML format into a plain text file. This will make the subsequent training faster and also allow easy inspection of the data = "input eyeballing".

We'll preprocess each article at the same time, normalizing its text to lowercase, splitting into tokens, etc. Below I use a regexp tokenizer that simply looks for alphabetic sequences as tokens. But feel free to adapt the text preprocessing to your own domain. High quality preprocessing is often critical for the final pipeline accuracy – garbage in, garbage out!

In [2]:
wiki = WikiCorpus(
    "enwiki-latest-pages-articles.xml.bz2",  # path to the file you downloaded above
    tokenizer_func=tokenize,  # simple regexp; plug in your own tokenizer here
    metadata=True,  # also return the article titles and ids when parsing
    dictionary={},  # don't start processing the data yet
)

with smart_open.open("wiki.txt.gz", "w", encoding='utf8') as fout:
    for article_no, (content, (page_id, title)) in enumerate(wiki.get_texts()):
        title = ' '.join(title.split())
        if article_no % 500000 == 0:
            logging.info("processing article #%i: %r (%i tokens)", article_no, title, len(content))
        fout.write(f"{title}\t{' '.join(content)}\n")  # title_of_article [TAB] words of the article

/Volumes/work/workspace/gensim/trunk/gensim/utils.py:1332: UserWarning: detected OSX with python3.8+; aliasing chunkize to chunkize_serial
  warnings.warn("detected %s; aliasing chunkize to chunkize_serial" % entity)
2022-03-17 21:15:32,118 : INFO : processing article #0: 'Anarchism' (6538 tokens)
2022-03-17 21:30:00,138 : INFO : processing article #500000: 'Spiritual Formation Bible' (54 tokens)
2022-03-17 21:40:22,219 : INFO : processing article #1000000: 'Adolf von Liebenberg' (52 tokens)
2022-03-17 21:49:43,825 : INFO : processing article #1500000: 'Small nucleolar RNA U6-53/MBII-28' (123 tokens)
2022-03-17 21:59:23,620 : INFO : processing article #2000000: 'Xie Fei' (50 tokens)
2022-03-17 22:09:17,460 : INFO : processing article #2500000: 'Rhein, Saskatchewan' (185 tokens)
2022-03-17 22:19:39,293 : INFO : processing article #3000000: 'Kunyinsky District' (969 tokens)
2022-03-17 22:30:41,221 : INFO : processing article #3500000: 'Lake Saint-Charles' (555 tokens)
2022-03-17 22:41:17

The above takes about 2 hours on my 2021 M1 MacbookPro, and creates a new ~5.8 GB file named `wiki.txt.gz`. We're compressing the text into `.gz` (GZIP) right away to save on disk space, using the [smart_open](https://github.com/RaRe-Technologies/smart_open) library.

Next we'll set up a stream to load the preprocessed articles from `wiki.txt.gz` one by one, in the format expected by Doc2Vec, ready for training. We don't want to load everything into RAM at once, because that would blow up the memory. And it is not necessary – Gensim can handle streamed training data:

In [3]:
class TaggedWikiCorpus:
    def __init__(self, wiki_text_path):
        self.wiki_text_path = wiki_text_path
        
    def __iter__(self):
        for line in smart_open.open(self.wiki_text_path, encoding='utf8'):
            title, words = line.split('\t')
            yield TaggedDocument(words=words.split(), tags=[title])

documents = TaggedWikiCorpus('wiki.txt.gz')  # A streamed iterable; nothing in RAM yet.

In [4]:
# Load and print the first preprocessed Wikipedia document, as a sanity check = "input eyeballing".
first_doc = next(iter(documents))
print(first_doc.tags, ': ', ' '.join(first_doc.words[:50] + ['………'] + first_doc.words[-50:]))

['Anarchism'] :  anarchism is political philosophy and movement that is sceptical of authority and rejects all involuntary coercive forms of hierarchy anarchism calls for the abolition of the state which it holds to be unnecessary undesirable and harmful as historically left wing movement placed on the farthest left of the political spectrum ……… criticism of philosophical anarchism defence of philosophical anarchism stating that both kinds of anarchism philosophical and political anarchism are philosophical and political claims anarchistic popular fiction novel an argument for philosophical anarchism external links anarchy archives anarchy archives is an online research center on the history and theory of anarchism


The document seems legit so let's move on to finally training some Doc2vec models.

## Training Doc2Vec

The original paper had a vocabulary size of 915,715 word types, so we'll try to match it by setting `max_final_vocab` to 1,000,000 in the Doc2vec constructor.

Other critical parameters were left unspecified in the paper, so we'll go with a default window size of five (a prediction window of 5 tokens to either side), and downsampling of frequent words at 1e-5. It looks like the authors tried vector dimensionality of 100, 300, 1,000 & 10,000 in the paper (with 10k dims performing the best), but I'll only train with 200 dimensions here, to keep RAM in check on my laptop.

Feel free to tinker with these values yourself if you like:

In [5]:
workers = multiprocessing.cpu_count()  # train with 10 threads on my 10-core laptop

# PV-DBOW: paragraph vector in distributed bag of words mode
model_dbow = Doc2Vec(
    dm=0, dbow_words=1,  # dbow_words=1 to train word vectors at the same time too, not only DBOW
    vector_size=200, window=8, sample=1e-5, epochs=10, workers=workers, max_final_vocab=1000000,
)

# PV-DM: paragraph vector in distributed memory mode
model_dm = Doc2Vec(
    dm=1, dm_concat=0, dm_mean=1,  # use average of context word vectors to train DM
    vector_size=200, window=8, sample=1e-5, epochs=10, workers=workers, max_final_vocab=1000000,
)

2022-03-23 11:46:26,539 : INFO : Doc2Vec lifecycle event {'params': 'Doc2Vec<dbow+w,d200,n5,w8,mc5,s1e-05,t10>', 'datetime': '2022-03-23T11:46:26.539501', 'gensim': '4.1.3.dev0', 'python': '3.8.9 (default, Oct 26 2021, 07:25:53) \n[Clang 13.0.0 (clang-1300.0.29.30)]', 'platform': 'macOS-12.2.1-arm64-arm-64bit', 'event': 'created'}
2022-03-23 11:46:26,541 : INFO : Doc2Vec lifecycle event {'params': 'Doc2Vec<dm/m,d200,n5,w8,mc5,s1e-05,t10>', 'datetime': '2022-03-23T11:46:26.541772', 'gensim': '4.1.3.dev0', 'python': '3.8.9 (default, Oct 26 2021, 07:25:53) \n[Clang 13.0.0 (clang-1300.0.29.30)]', 'platform': 'macOS-12.2.1-arm64-arm-64bit', 'event': 'created'}


In [6]:
model_dbow.build_vocab(documents, progress_per=500000)
print(model_dbow)

# Save some time by copying the vocabulary structures from the DBOW model to the DM model.
# Both models are built on top of exactly the same data, so there's no need to repeat the vocab-building step.
model_dm.reset_from(model_dbow)
print(model_dm)

2022-03-23 11:46:26,921 : INFO : collecting all words and their counts
2022-03-23 11:46:26,926 : INFO : PROGRESS: at example #0, processed 0 words (0 words/s), 0 word types, 0 tags
2022-03-23 11:48:29,646 : INFO : PROGRESS: at example #500000, processed 654950164 words (5336561 words/s), 3222179 word types, 500000 tags
2022-03-23 11:49:40,672 : INFO : PROGRESS: at example #1000000, processed 1018611068 words (5119067 words/s), 4480366 word types, 1000000 tags
2022-03-23 11:50:36,816 : INFO : PROGRESS: at example #1500000, processed 1305140647 words (5103506 words/s), 5420104 word types, 1500000 tags
2022-03-23 11:51:25,894 : INFO : PROGRESS: at example #2000000, processed 1550245240 words (4994178 words/s), 6188355 word types, 2000000 tags
2022-03-23 11:52:14,324 : INFO : PROGRESS: at example #2500000, processed 1790661139 words (4964223 words/s), 6941128 word types, 2500000 tags
2022-03-23 11:53:02,465 : INFO : PROGRESS: at example #3000000, processed 2028261627 words (4935656 words/s

Doc2Vec<dbow+w,d200,n5,w8,mc5,s1e-05,t10>
Doc2Vec<dm/m,d200,n5,w8,mc5,s1e-05,t10>


Now we’re ready to train Doc2Vec on the entirety of the English Wikipedia. **Warning!** Training this DBOW model takes ~9 hours, and DM ~4 hours, on my 2021 laptop.

In [7]:
# Train DBOW doc2vec incl. word vectors
model_dbow.train(documents, total_examples=model_dbow.corpus_count, epochs=model_dbow.epochs, report_delay=30*60)

2022-03-23 11:57:10,178 : INFO : Doc2Vec lifecycle event {'msg': 'training model with 10 workers on 991887 vocabulary and 200 features, using sg=1 hs=0 sample=1e-05 negative=5 window=8 shrink_windows=True', 'datetime': '2022-03-23T11:57:10.178111', 'gensim': '4.1.3.dev0', 'python': '3.8.9 (default, Oct 26 2021, 07:25:53) \n[Clang 13.0.0 (clang-1300.0.29.30)]', 'platform': 'macOS-12.2.1-arm64-arm-64bit', 'event': 'train'}
2022-03-23 11:57:11,217 : INFO : EPOCH 0 - PROGRESS: at 0.00% examples, 2586 words/s, in_qsize 8, out_qsize 0
2022-03-23 12:27:11,295 : INFO : EPOCH 0 - PROGRESS: at 45.70% examples, 379076 words/s, in_qsize 20, out_qsize 0
2022-03-23 12:48:46,182 : INFO : EPOCH 0: training on 2996051328 raw words (1198675664 effective words) took 3096.0s, 387174 effective words/s
2022-03-23 12:48:47,191 : INFO : EPOCH 1 - PROGRESS: at 0.00% examples, 407128 words/s, in_qsize 19, out_qsize 0
2022-03-23 13:18:47,205 : INFO : EPOCH 1 - PROGRESS: at 48.78% examples, 396088 words/s, in_qsi

In [8]:
# Train DM doc2vec
model_dm.train(documents, total_examples=model_dm.corpus_count, epochs=model_dm.epochs, report_delay=30*60)

2022-03-23 21:06:50,772 : INFO : Doc2Vec lifecycle event {'msg': 'training model with 10 workers on 991887 vocabulary and 200 features, using sg=0 hs=0 sample=1e-05 negative=5 window=8 shrink_windows=True', 'datetime': '2022-03-23T21:06:50.772480', 'gensim': '4.1.3.dev0', 'python': '3.8.9 (default, Oct 26 2021, 07:25:53) \n[Clang 13.0.0 (clang-1300.0.29.30)]', 'platform': 'macOS-12.2.1-arm64-arm-64bit', 'event': 'train'}
2022-03-23 21:06:51,779 : INFO : EPOCH 0 - PROGRESS: at 0.01% examples, 774441 words/s, in_qsize 0, out_qsize 0
2022-03-23 21:28:47,548 : INFO : EPOCH 0: training on 2996051328 raw words (1198677606 effective words) took 1316.7s, 910333 effective words/s
2022-03-23 21:28:48,576 : INFO : EPOCH 1 - PROGRESS: at 0.02% examples, 1133718 words/s, in_qsize 0, out_qsize 0


KeyboardInterrupt: 

## Similarity interface

After that, let's test both models! The DBOW model shows similar results as the original paper.

First, calculate the most similar Wikipedia articles to the "Machine learning" article. The calculated word vectors and document vectors are stored separately, in `model.wv` and `model.dv` respectively:

In [9]:
for model in [model_dbow, model_dm]:
    print(model)
    pprint(model.dv.most_similar(positive=["Machine learning"], topn=20))

Doc2Vec<dbow+w,d200,n5,w8,mc5,s1e-05,t10>
[('Pattern recognition', 0.7641374468803406),
 ('Multi-task learning', 0.7290244698524475),
 ('Supervised learning', 0.7212514877319336),
 ('Incremental learning', 0.7164462208747864),
 ('Deep learning', 0.7093881964683533),
 ('Predictive analytics', 0.7086609601974487),
 ('Semi-supervised learning', 0.7068915367126465),
 ('Outline of machine learning', 0.7035143971443176),
 ('Artificial neural network', 0.6998467445373535),
 ('Ensemble learning', 0.6948938965797424),
 ('Intelligent control', 0.6883038878440857),
 ('Statistical classification', 0.6876234412193298),
 ('Rule induction', 0.6867162585258484),
 ('Boosting (machine learning)', 0.685867190361023),
 ('Feature selection', 0.6836000084877014),
 ('Training, validation, and test sets', 0.6823415160179138),
 ('Support-vector machine', 0.6810059547424316),
 ('Perceptron', 0.6794257760047913),
 ('Multilayer perceptron', 0.6773776412010193),
 ('Neural network', 0.6765708923339844)]
Doc2Vec<dm/

Both results seem similar and match the results from the paper's Table 1, although not exactly. This is because we don't know the exact parameters of the original implementation (see above). And also because we're training the model 7 years later and the Wikipedia content has changed in the meantime.

Now following the paper's Table 2a), let's calculate the most similar Wikipedia entries to "Lady Gaga" using Paragraph Vector:

In [10]:
for model in [model_dbow, model_dm]:
    print(model)
    pprint(model.dv.most_similar(positive=["Lady Gaga"], topn=10))

Doc2Vec<dbow+w,d200,n5,w8,mc5,s1e-05,t10>
[('Katy Perry', 0.7140653133392334),
 ('Ariana Grande', 0.6990166306495667),
 ('Demi Lovato', 0.6782864332199097),
 ('Miley Cyrus', 0.6620475053787231),
 ('List of awards and nominations received by Lady Gaga', 0.6562342047691345),
 ('Christina Aguilera', 0.6527020335197449),
 ('Taylor Swift', 0.6430284380912781),
 ('Adele', 0.6412620544433594),
 ('Adam Lambert', 0.6401858329772949),
 ('Halsey (singer)', 0.637832760810852)]
Doc2Vec<dm/m,d200,n5,w8,mc5,s1e-05,t10>
[('Katy Perry', 0.6719839572906494),
 ('Ariana Grande', 0.6502904295921326),
 ('Taylor Swift', 0.6452381014823914),
 ('Artpop', 0.6417931914329529),
 ('Christina Aguilera', 0.634290337562561),
 ('Nicki Minaj', 0.6294941902160645),
 ('Adam Lambert', 0.6128465533256531),
 ('Kesha', 0.6105154156684875),
 ('Born This Way (album)', 0.6087599992752075),
 ('Adele', 0.6087093353271484)]


The DBOW results are in line with what the paper shows in Table 2a), revealing similar singers in the U.S.

Finally, let's do some of the wilder arithmetics that vectors embeddings are famous for. What are the entries most similar to "Lady Gaga" - "American" + "Japanese"? Table 2b) in the paper.

Note that "American" and "Japanese" are word vectors, but they live in the same space as the document vectors so we can add / subtract them at will, for some interesting results. All word vectors were already lowercased by our tokenizer above, so we look for the lowercased version here:

In [11]:
for model in [model_dbow, model_dm]:
    print(model)
    vec = [model.dv["Lady Gaga"] - model.wv["american"] + model.wv["japanese"]]
    pprint([m for m in model.dv.most_similar(vec, topn=11) if m[0] != "Lady Gaga"])

Doc2Vec<dbow+w,d200,n5,w8,mc5,s1e-05,t10>
[('Katy Perry', 0.6580742001533508),
 ('Kōsui (Eito song)', 0.6197645664215088),
 ('2NE1', 0.6172165274620056),
 ('Ariana Grande', 0.608268678188324),
 ('Alex York', 0.5975368618965149),
 ('Thank You, Love (Kana Nishino album)', 0.5951482653617859),
 ('X -Cross-', 0.5949676632881165),
 ('Megitsune', 0.5922212600708008),
 ('7 Spirits', 0.5915307998657227),
 ('Audience (Ayumi Hamasaki song)', 0.5913636088371277)]
Doc2Vec<dm/m,d200,n5,w8,mc5,s1e-05,t10>
[('Morning Musume', 0.6124014854431152),
 ('Yuko Ando (singer)', 0.6063645482063293),
 ('Yumi Matsutoya', 0.6047919392585754),
 ('J-pop', 0.5908822417259216),
 ('Ayumi Hamasaki', 0.5900821685791016),
 ('E-girls', 0.5884340405464172),
 ('Enka', 0.583469033241272),
 ('Shingo Katori', 0.583054780960083),
 ('Dempagumi.inc', 0.575444221496582),
 ('Shinsei Kamattechan', 0.5742727518081665)]


As a result, the DBOW model surfaced artists similar to Lady Gaga in Japan, such as **Ayumi Hamasaki** whose Wiki bio says:

> Ayumi Hamasaki is a Japanese singer, songwriter, record producer, actress, model, spokesperson, and entrepreneur.

So that sounds like a success. It's also the nr. 1 hit in the paper we're replicating.

Similarly, the DM model thought **Kaela Kimura** is the closest hit:

> Kaela Kimura is a Japanese pop rock singer, lyricist, fashion model and television presenter.

Also pretty good.

These results demonstrate that both training modes employed in the original paper are outstanding for calculating similarity between document vectors, word vectors, or a combination of both. The DM mode has the added advantage of being 4x faster to train.

If you wanted to continue working with these trained models, you could save them to disk, to avoid having to re-train the models from scratch every time:

In [ ]:
model_dbow.save('doc2vec_dbow.model')
model_dm.save('doc2vec_dm.model')

To continue your doc2vec explorations, refer to the official API documentation in Gensim: https://radimrehurek.com/gensim/models/doc2vec.html